In [ ]:
import cv2 as cv

In [ ]:
# cap = cv.VideoCapture(0)

# while True:
    
#     ret,frame = cap.read()
    
#     cv.imshow("frame",frame)
    
#     if cv.waitKey(2) & 0xFF == ord('x'):
#         break
# cap.release()
# cv.destroyAllWindows()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from utils.Inceptionresnet import *

In [ ]:
from keras.applications import InceptionResNetV2
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import tensorflow as tf
from keras import backend as K

In [ ]:
def load_inception_resnet(
    weights="imagenet", input_shape=(160, 160, 3), include_top=False
):
    """
    Load the InceptionResNetV2 model with specific configurations.
    :param weights: Pre-trained weights or path to custom weights.
    :param input_shape: Input image shape (default is (160, 160, 3)).
    :param include_top: Whether to include the top layers.
    :return: InceptionResNetV2 model instance.
    """
    return InceptionResNetV2(
        weights=weights, input_shape=input_shape, include_top=include_top
    )


# Initialize the model
facenet_encoder = load_inception_resnet()
facenet_encoder.load_weights("utils/facenet_keras_weights.h5", by_name=True)
FRmodel = facenet_encoder

In [ ]:
print(FRmodel.inputs)
print(FRmodel.outputs)

In [ ]:
FRmodel.summary()

In [ ]:
from skimage.transform import resize


def img_to_encoding(image, model):
    if image is None:
        return 0
    else:
        img = cv.resize(image,(160,160))
        img = np.around(np.asarray(img)/255.0, decimals=12)
        x_train = np.expand_dims(img, axis=0)
        embedding = model.predict_on_batch(x_train)
        return embedding / np.linalg.norm(embedding, ord=2)

In [ ]:
def img_to_encoding(image, model):
    """
    Converts the input image to an encoding using the given model.

    Arguments:
    image -- the input image
    model -- the face recognition model

    Returns:
    encoding -- the normalized face encoding
    """
    # Resize the image to the expected input size (160x160)
    img = cv.resize(image, (160, 160))
    
    # Normalize pixel values between 0 and 1
    img = np.around(np.asarray(img) / 255.0, decimals=12)

    # Get the embedding from the model
    embedding = model.predict_on_batch(np.expand_dims(img, axis=0))

    # Flatten the embedding to ensure it's a 1D array
    embedding = embedding.flatten()

    # Print the shape of the embedding to understand its dimensions
    print("Embedding shape:", embedding.shape)

    # Normalize the embedding vector
    norm_embedding = np.linalg.norm(embedding, ord=2)

    # If norm is zero, return a zero embedding or handle it gracefully
    if norm_embedding == 0:
        print("Warning: Zero embedding detected")
        return embedding  # Return the original embedding, or handle as needed

    return embedding / norm_embedding


# Load images and create a database of encodings
filepath = "extract_face/"
database = {}


def load_image(filepath):
    """
    Load images from the specified directory and encode them into the database.
    :param filepath: Directory containing folders of images.
    :return: A dictionary where keys are folder names (identities) and values are lists of encodings.
    """
    for folder in os.listdir(filepath):
        encodings = []
        for subfolder in os.listdir(filepath + folder):
            image_BGR = cv.imread(filepath + folder + "/" + subfolder)
            if image_BGR is not None:
                temp = img_to_encoding(image_BGR, FRmodel)
                encodings.append(temp)
        database[folder] = encodings
    return database


# Load the image database
database = load_image(filepath)

In [ ]:
database["Ketul"][10].shape

In [ ]:
print(database.items())

# Face Verification

In [ ]:
#face verification
def verify(image, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
        image_path -- path to an image
        identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
        database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
        model -- your Inception model instance in Keras
    
    Returns:
        dist -- distance between the image_path and the image of "identity" in the database.
        door_open -- True, if the door should open. False otherwise.
    """
    ### START CODE HERE
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
#     image_RGB = cv.cvtColor(image,cv.COLOR_BGR2RGB)
#     pixel = detector.detect_faces(image_RGB)
#     x,y,w,h = pixel[0]['box']
#     img = image_RGB[y:y+h,x:x+w]
    encoding = img_to_encoding(image,model)
    for i in database[identity]:
        # Step 2: Compute distance with identity's image (≈ 1 line)
        print(i)
        dist = np.linalg.norm(encoding - i)
        # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
        if dist<0.6:
            print("It's " + str(identity) + ", welcome in!")
            door_open = True
            break
        else:
            print("It's not " + str(identity) + ", please go away")
            door_open = False
            break
    ### END CODE HERE        
    return dist, door_open,encoding

In [ ]:
distance, door_open_flag, encoding = verify(cv.imread("extract_face/Ketul/IMG20241127160112_BURST005_32714670.jpg"), "Ketul", database, FRmodel)
print("(", distance, ",", door_open_flag, ")")
print(encoding.shape)

# Face Recognition

In [ ]:
def who_is_it(image, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.

    Arguments:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras

    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """

    # Step 1: Compute the target "encoding" for the image
    encoding = img_to_encoding(image, model)

    # Step 2: Find the closest encoding
    min_dist = 100  # Initialize "min_dist" to a large value
    identity = (
        None  # Initialize identity to None, to handle case where no match is found
    )

    # Loop over the database dictionary's names and encodings
    for name, db_enc in database.items():
        for i in db_enc:
            # Compute L2 distance between the target "encoding" and the current db_enc from the database
            dist = np.linalg.norm(encoding - i)

            # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
            if dist < min_dist:
                min_dist = dist
                identity = name  # Set identity to the matching name

    # If no match was found (identity is None), handle it
    if identity is None:
        print("No match found!")
    else:
        if min_dist > 0.55:
            print("No match with sufficient confidence")
        else:
            print(f"It's {identity}, the distance is {min_dist}")

    return min_dist, identity

In [ ]:
# image_RGB = cv.cvtColor(cv.imread("Data/val/jay/12.jpg"),cv.COLOR_BGR2RGB)
# pixel = detector.detect_faces(image_RGB)
# x,y,w,h = pixel[0]['box']
# img = image_RGB[y:y+h,x:x+w]
img = cv.imread("extract_face/Ketul/IMG20241127160112_BURST005_32714670.jpg")
# color = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# face = haar_casecade.detectMultiScale(color,
#                                           scaleFactor=1.3,
#                                          minNeighbors=5,)
# for (x,y,w,h) in face:
#     image = img[y:y+h,x:x+w]
test1 = who_is_it(img, database, FRmodel)

#  Face Detection

In [ ]:
haar_casecade = cv.CascadeClassifier("utils/haar_face.xml")

In [ ]:
!pip install mtcnn

In [ ]:
from mtcnn import MTCNN

In [ ]:
detector = MTCNN()

In [ ]:
!py -3.9 -m pip install face_recognition
import face_recognition


In [ ]:
import dlib

# Face Detection and Recognition

In [ ]:
cap = cv.VideoCapture(0)
# fourcc = cv.VideoWriter_fourcc(*'XVID')
# out = cv.VideoWriter("output.mp4",fourcc,10.00,(640,480))

while True:
    content,frame = cap.read()
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
#     image_RGB = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    faces = haar_casecade.detectMultiScale(gray,
                                          scaleFactor=1.3,
                                          minNeighbors=4,
                                        )
    
#     face = detector.detect_faces(image_RGB)
#     if face != []:
#         box = np.array([face[0]['box']])
    
#     face_detector = dlib.get_frontal_face_detector()
#     landmark_predictor = dlib.shape_predictor("utils/shape_predictor_68_face_landmarks.dat")
    
    #faces = face_detector(gray)

    for (x,y,w,h) in faces:
        
        #x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),thickness=2)
        # Extract the face region
        face1 = frame[y:y+h, x:x+w]
        
        
        # Perform face alignment using facial landmarks
        face_locations = face_recognition.face_locations(frame)
        face_landmarks = face_recognition.face_landmarks(frame, face_locations)
        #print(face_landmarks)

#         for landmarks in face_landmarks:
#             # Get coordinates of left eye, right eye, and nose tip
#             left_eye = landmarks['left_eye']
#             right_eye = landmarks['right_eye']
#             nose_tip = landmarks['nose_tip']

#             # Calculate the center of mass for the eyes
#             left_eye_center = np.mean(left_eye, axis=0).astype(int)
#             right_eye_center = np.mean(right_eye, axis=0).astype(int)

#             # Calculate the angle between the eyes
#             angle = np.degrees(np.arctan2(right_eye_center[1] - left_eye_center[1], right_eye_center[0] - left_eye_center[0]))

#             # Perform rotation to align the face
#             rotation_matrix = cv.getRotationMatrix2D(tuple(nose_tip[0]), angle, scale=1)
#             aligned_face = cv.warpAffine(face, rotation_matrix, (w, h), flags=cv.INTER_LINEAR)

            
        mindist,identity = who_is_it(face1,database,FRmodel)

        if mindist<0.6:
            cv.putText(frame,identity,(x,y-10),cv.FONT_HERSHEY_PLAIN,2,(0,255,0),2,cv.LINE_AA)           
#         else:
#             cv.putText(frame,'Unknown',(x,y-10),cv.FONT_HERSHEY_PLAIN,2,(0,0,255),2,cv.LINE_AA)
            
    #out.write(frame)


    cv.imshow("video",frame)
    
    if cv.waitKey(2) & 0xFF == ord('x'):
        break

cap.release()
#out.release()
cv.destroyAllWindows()

# Face Detection Using Mediapipe

In [ ]:
import time
import cv2 as cv
import mediapipe as mp
import numpy as np

mp_face_detection = mp.solutions.face_detection
cap = cv.VideoCapture(0)
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detector:
    frame_counter = 0
    fonts = cv.FONT_HERSHEY_PLAIN
    start_time = time.time()
    while True:
        frame_counter += 1
        ret, frame = cap.read()
        if ret is False:
            break
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        results = face_detector.process(rgb_frame)
        frame_height, frame_width, c = frame.shape
        if results.detections:
            for face in results.detections:
                face_react = np.multiply(
                    [
                        face.location_data.relative_bounding_box.xmin,
                        face.location_data.relative_bounding_box.ymin,
                        face.location_data.relative_bounding_box.width,
                        face.location_data.relative_bounding_box.height,
                    ],
                    [frame_width, frame_height, frame_width, frame_height]).astype(int)
                #print(face_react)
                x = face_react[0]
                y = face_react[1]
                w = face_react[2]
                h = face_react[3]
                face = frame[y:y+h,x:x+w]
                
                cv.rectangle(frame, face_react, color=(255, 255, 255), thickness=2)
#                 key_points = np.array([(p.x, p.y) for p in face.location_data.relative_keypoints])
#                 key_points_coords = np.multiply(key_points,[frame_width,frame_height]).astype(int)
#                 for p in key_points_coords:
#                     cv.circle(frame, p, 4, (255, 255, 255), 2)
#                     cv.circle(frame, p, 2, (0, 0, 0), -1)
                mindist,identity = who_is_it(face,database,FRmodel)

                if mindist<0.55:
                    cv.putText(frame,identity,(x,y-10),cv.FONT_HERSHEY_PLAIN,2,(0,255,0),2,cv.LINE_AA)           
#                 else:
#                     cv.putText(frame,'Unknown',(x,y-10),cv.FONT_HERSHEY_PLAIN,2,(0,0,255),2,cv.LINE_AA)
        
        fps = frame_counter / (time.time() - start_time)
        cv.putText(frame,f"FPS: {fps:.2f}",(30, 30),cv.FONT_HERSHEY_DUPLEX,0.7,(0, 255, 255),2,)
        cv.imshow("frame", frame)
        
        if cv.waitKey(1) & 0xFF == ord("x"):
            break
    cap.release()
    cv.destroyAllWindows() 

In [ ]:
#save model

FRmodel.save("saved_model.h5")

In [ ]:
#convert into tflite

import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the Keras model
loaded_model = load_model('saved_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('saved_model.tflite', 'wb') as f:
    f.write(tflite_model)
